## MASTER DATA SCIENCE: NUCLIO
## PROFESOR: JOSEPH GALLART
## PROJECT RENFE: CREAMOS EL MODELO QUE PREDICE EL PRECIO DEL BILLETE

In [1]:
#importamos librería
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math


from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor

import pickle

from sklearn import metrics
from scipy import stats



In [2]:
fichero=open('datasets/dataset_renfe_final','rb')
df_renfe_final=pickle.load(fichero)

df_renfe_final.shape

(354505, 52)

In [3]:
df_renfe_final.head()

,PRECIO,TIEMPO_VIAJE,DIF_INI_BUS,FECHA_INICIO_HORA,FECHA_INICIO_DIA,FECHA_INICIO_MES,FECHA_FIN_HORA,LOG_PRECIO,CIUDAD_ORIGEN_MADRID,CIUDAD_ORIGEN_PONFERRADA,...,CLASE_Turista Plus,CLASE_Turista PlusSólo plaza H,CLASE_Turista con enlace,CLASE_TuristaSólo plaza H,FECHA_INICIO_NOMBREDIA_Monday,FECHA_INICIO_NOMBREDIA_Saturday,FECHA_INICIO_NOMBREDIA_Sunday,FECHA_INICIO_NOMBREDIA_Thursday,FECHA_INICIO_NOMBREDIA_Tuesday,FECHA_INICIO_NOMBREDIA_Wednesday
0,67.20,0.092736,0.581605,0.90,0.900000,0.333333,1.000000,1.827369,0,0,...,0,0,0,0,0,0,0,0,0,0
1,107.70,0.142195,0.365538,0.40,0.433333,0.500000,0.565217,2.032216,0,0,...,0,0,0,0,0,0,1,0,0,0
2,47.30,0.080371,0.606966,0.55,0.733333,0.166667,0.695652,1.674861,0,0,...,0,0,0,0,0,0,0,1,0,0
3,28.35,0.559505,0.312947,0.60,0.933333,0.166667,0.956522,1.452553,0,0,...,0,0,0,0,0,0,0,0,0,1
4,66.75,0.125193,0.798256,0.55,0.833333,0.333333,0.695652,1.824451,1,0,...,0,0,0,0,0,0,0,0,0,1


In [4]:

### VAMOS APLICAR LA REGRESION LINEAL Y EL RANDOM FOREST REGRESSION, PARA ESTE NO ES NECESARIO CONVERTIRLO AL LOG EL PRECIO
def create_results_df(model, X_test, y_test):
    print('Construyendo DataFrame de resultados')
    y_test.columns = ['Label']
    test_prediction = pd.DataFrame(model.predict(X_test), columns=['Prediction'], index=X_test.index)
    results_df = y_test.join(test_prediction)
    results_df['squared_error'] = (results_df['Label'] - results_df['Prediction'])**2
    results_df['rooted_squared_error'] = np.sqrt(results_df['squared_error'])
    mse = results_df['squared_error'].mean()
    rmse = results_df['rooted_squared_error'].mean()
    mae = rmse / y_test['Label'].mean()
    accuracy = model.score(X_test,y_test)
    print('MSE: {}'.format(mse))
    print('RMSE: {}'.format(rmse))
    print('MAE: {}'.format(mae))
    print('Accuracy: {}'.format(accuracy))
    return results_df

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
df_renfe_final.head()

,PRECIO,TIEMPO_VIAJE,DIF_INI_BUS,FECHA_INICIO_HORA,FECHA_INICIO_DIA,FECHA_INICIO_MES,FECHA_FIN_HORA,LOG_PRECIO,CIUDAD_ORIGEN_MADRID,CIUDAD_ORIGEN_PONFERRADA,...,CLASE_Turista Plus,CLASE_Turista PlusSólo plaza H,CLASE_Turista con enlace,CLASE_TuristaSólo plaza H,FECHA_INICIO_NOMBREDIA_Monday,FECHA_INICIO_NOMBREDIA_Saturday,FECHA_INICIO_NOMBREDIA_Sunday,FECHA_INICIO_NOMBREDIA_Thursday,FECHA_INICIO_NOMBREDIA_Tuesday,FECHA_INICIO_NOMBREDIA_Wednesday
0,67.20,0.092736,0.581605,0.90,0.900000,0.333333,1.000000,1.827369,0,0,...,0,0,0,0,0,0,0,0,0,0
1,107.70,0.142195,0.365538,0.40,0.433333,0.500000,0.565217,2.032216,0,0,...,0,0,0,0,0,0,1,0,0,0
2,47.30,0.080371,0.606966,0.55,0.733333,0.166667,0.695652,1.674861,0,0,...,0,0,0,0,0,0,0,1,0,0
3,28.35,0.559505,0.312947,0.60,0.933333,0.166667,0.956522,1.452553,0,0,...,0,0,0,0,0,0,0,0,0,1
4,66.75,0.125193,0.798256,0.55,0.833333,0.333333,0.695652,1.824451,1,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
X = df_renfe_final.drop(['LOG_PRECIO','PRECIO'],axis=1) # INDEPENDIENTES VARIABLES

In [8]:
y = df_renfe_final[['PRECIO']] #  VARIABLES DEPENDIENTE (A PREDECIR)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=23)

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
lin_reg= LinearRegression()

In [12]:
lin_reg.fit(X_train,y_train)

LinearRegression()

In [13]:
resultados=create_results_df(lin_reg,X_test,y_test)

Construyendo DataFrame de resultados
MSE: 100.83351545970153
RMSE: 7.191225948089902
MAE: 0.11719208336286595
Accuracy: 0.8460988190392568


In [14]:
### Ahora con Random Forest
rf = RandomForestRegressor(n_estimators= 10, random_state=42)

In [15]:
rf.fit(X_train, y_train.values.ravel())

RandomForestRegressor(n_estimators=10, random_state=42)

In [16]:
resultados_rf=create_results_df(rf,X_test,y_test)

Construyendo DataFrame de resultados
MSE: 10.874963338068385
RMSE: 1.2842936020637463
MAE: 0.020929538852193542
Accuracy: 0.9834016527837673


In [17]:
feature_list=X_train.columns

In [18]:
# Lista de variables
importances = list(rf.feature_importances_)

In [19]:
feature_list_importance=[]
for feature, importance in zip(feature_list, importances):
    feature_list_importance.append([feature,round(importance,2)])

In [20]:
feature_list_importance.sort(key=lambda x:x[1],reverse=True)

In [51]:
feature_list_importance

[['TIEMPO_VIAJE', 0.42],
 ['TIPO_TARIFA_Flexible', 0.17],
 ['CIUDAD_DESTINO_SEVILLA', 0.11],
 ['CIUDAD_ORIGEN_SEVILLA', 0.08],
 ['DIF_INI_BUS', 0.03],
 ['CLASE_Preferente', 0.03],
 ['CLASE_Turista', 0.03],
 ['FECHA_INICIO_HORA', 0.02],
 ['FECHA_INICIO_MES', 0.02],
 ['FECHA_FIN_HORA', 0.02],
 ['FECHA_INICIO_DIA', 0.01],
 ['CIUDAD_ORIGEN_VALENCIA', 0.01],
 ['TIPO_TREN_AVE', 0.01],
 ['CIUDAD_ORIGEN_MADRID', 0.0],
 ['CIUDAD_ORIGEN_PONFERRADA', 0.0],
 ['CIUDAD_DESTINO_MADRID', 0.0],
 ['CIUDAD_DESTINO_PONFERRADA', 0.0],
 ['CIUDAD_DESTINO_VALENCIA', 0.0],
 ['TIPO_TREN_AV City', 0.0],
 ['TIPO_TREN_AVE-LD', 0.0],
 ['TIPO_TREN_AVE-MD', 0.0],
 ['TIPO_TREN_AVE-TGV', 0.0],
 ['TIPO_TREN_INTERCITY', 0.0],
 ['TIPO_TREN_LD', 0.0],
 ['TIPO_TREN_LD-MD', 0.0],
 ['TIPO_TREN_MD', 0.0],
 ['TIPO_TREN_MD-AVE', 0.0],
 ['TIPO_TREN_MD-LD', 0.0],
 ['TIPO_TREN_R. EXPRES', 0.0],
 ['TIPO_TREN_REGIONAL', 0.0],
 ['TIPO_TREN_TRENHOTEL', 0.0],
 ['TIPO_TARIFA_COD.PROMOCIONAL', 0.0],
 ['TIPO_TARIFA_Doble Familiar-Flexible'

In [22]:
## Vamos a hacer el modelo con menos variables
rf2 = RandomForestRegressor(n_estimators= 10, random_state=42)

In [57]:
type(feature_list_importance)

list

In [ ]:
variables_importantes=[]
for i in feature_list_importance:
    if (i[1]>=0.0172) and (i[0]!="FECHA_INICIO_MES") and (i[0]!="FECHA_FIN_HORA"):
        variables_importantes.append(i[0])

In [87]:
variables_importantes

['TIEMPO_VIAJE',
 'TIPO_TARIFA_Flexible',
 'CIUDAD_DESTINO_SEVILLA',
 'CIUDAD_ORIGEN_SEVILLA',
 'DIF_INI_BUS',
 'CLASE_Preferente',
 'CLASE_Turista',
 'FECHA_INICIO_HORA']

In [88]:
X_train2=X_train[variables_importantes]

In [89]:
X_test2=X_test[variables_importantes]

In [90]:
rf2.fit(X_train2, y_train.values.ravel())

RandomForestRegressor(n_estimators=10, random_state=42)

In [91]:
resultados2=create_results_df(rf2,X_test2,y_test)

Construyendo DataFrame de resultados
MSE: 81.78193480397843
RMSE: 5.564055882822798
MAE: 0.09067484536883615
Accuracy: 0.875177055067397


In [92]:
outfile= open("modelos/modelo_rf_final", "wb")

In [93]:
pickle.dump(rf2,outfile)

In [94]:
outfile.close()

In [95]:
lista_variables=list(X_train2.columns)

In [96]:
variables_importantes

['TIEMPO_VIAJE',
 'TIPO_TARIFA_Flexible',
 'CIUDAD_DESTINO_SEVILLA',
 'CIUDAD_ORIGEN_SEVILLA',
 'DIF_INI_BUS',
 'CLASE_Preferente',
 'CLASE_Turista',
 'FECHA_INICIO_HORA']

In [97]:
outfile= open("modelos/variables_importantes", "wb")

In [98]:
pickle.dump(variables_importantes,outfile)

In [99]:
outfile.close()

In [100]:
variables_importantes

['TIEMPO_VIAJE',
 'TIPO_TARIFA_Flexible',
 'CIUDAD_DESTINO_SEVILLA',
 'CIUDAD_ORIGEN_SEVILLA',
 'DIF_INI_BUS',
 'CLASE_Preferente',
 'CLASE_Turista',
 'FECHA_INICIO_HORA']

In [101]:
#Estas 10 variables serán la entrada de nuestro modelo para que pueda predecir el precio

#TIEMPO_VIAJE: no se puede pedir al cliente debemos nosotros calcularlo
#TIPO_TARIFA_Flexible, fare_Promo: son opciones que debemos mostrar como resultado al cliente
#CIUDAD_ORIGEN_SEVILLA, CIUDAD_ORIGEN_SEVILLA: opciones de ciudad que debemos mostrar
#DIF_INI_BUS: debemos calcularlo (para ello debemos pedirle la fecha de viaje)
#CLASE_Preferente, CLASE_Turista: igual que el fare debemos mostrar como resultado al cliente
#FECHA_INICIO_HORA,FECHA_INICIO_MES: debemos pedirlo
#FECHA_FIN_HORA: no debemos pedirlo debemos mostrar todas las posibilidad de cálculo de viaje que tiene nuestro dataframe

In [102]:
fichero=open('datasets/dataset_renfe_preprocessing','rb')

In [103]:
df_renfe_prep=pickle.load(fichero)

In [111]:
df_renfe_prep.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354505 entries, 0 to 354504
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   CIUDAD_ORIGEN           354505 non-null  object 
 1   CIUDAD_DESTINO          354505 non-null  object 
 2   TIPO_TREN               354505 non-null  object 
 3   TIPO_TARIFA             354505 non-null  object 
 4   CLASE                   354505 non-null  object 
 5   PRECIO                  354505 non-null  float64
 6   TIEMPO_VIAJE            354505 non-null  float64
 7   DIF_INI_BUS             354505 non-null  float64
 8   FECHA_INICIO_HORA       354505 non-null  int64  
 9   FECHA_INICIO_NOMBREDIA  354505 non-null  object 
 10  FECHA_INICIO_DIA        354505 non-null  int64  
 11  FECHA_INICIO_MES        354505 non-null  int64  
 12  FECHA_FIN_HORA          354505 non-null  int64  
 13  FECHA_FIN_NOMBREDIA     354505 non-null  object 
 14  LOG_PRECIO          

In [105]:
#VAMOS A CREAR LA TABLA DF_TIEMPO_DESTINO QUE NOS PERMITIRA VER TODOS LOS TIEMPOS MEDIOS QUE TENEMOS EN NUESTRO DATAFRAME
#DADA UNA CIUDAD ORIGEN, CIUDAD DESTINO Y TIPO DE TREN, CON EL OBJETIVO DE MOSTRAR AL CLIENTE TODAS LAS POSIBILIDADES DE VIAJE QUE TIENE
#EL TIPO DE TREN ES IMPORTANTE PARA MOSTRAR AL CLIENTE LAS POSIBILIDADES, MAS NO ENTRARA EL TIPO DE TREN EN EL MODELO
df_tiempo_destino=df_renfe_prep.groupby(["CIUDAD_ORIGEN","CIUDAD_DESTINO","TIPO_TREN"]).agg(TIEMPO_VIAJE=('TIEMPO_VIAJE', 'mean')).reset_index()

In [106]:
df_tiempo_destino

,CIUDAD_ORIGEN,CIUDAD_DESTINO,TIPO_TREN,TIEMPO_VIAJE
0,BARCELONA,MADRID,AVE,170.583315
1,BARCELONA,MADRID,AVE-TGV,175.000000
2,BARCELONA,MADRID,R. EXPRES,544.000000
3,MADRID,BARCELONA,AVE,170.251589
4,MADRID,BARCELONA,AVE-TGV,179.000000
5,MADRID,BARCELONA,R. EXPRES,562.000000
6,MADRID,PONFERRADA,ALVIA,243.000000
7,MADRID,PONFERRADA,AVE-LD,252.000000
8,MADRID,PONFERRADA,AVE-MD,245.714286
9,MADRID,PONFERRADA,LD,252.000000


In [107]:
outfile=open("datasets/df_tiempo_destino","wb")

In [108]:
pickle.dump(df_tiempo_destino,outfile)

In [109]:
outfile.close()